# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Column summary
| Variable | Definition                                 | Key                                            | int_value                                      |
|----------|--------------------------------------------|------------------------------------------------|------------------------------------------------|
| survival | Survival                                   | 0 = No, 1 = Yes                                |                                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |                                                |
| sex      | Sex                                        |                                                | 0 = male, 1 = female                           |
| Age      | Age in years                               |                                                |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |                                                |
| ticket   | Ticket number                              |                                                |                                                |
| fare     | Passenger fare                             |                                                |                                                |
| cabin    | Cabin number                               |                                                |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton | 0 = Cherbourg, 1 = Queenstown, 2 = Southampton |

# Load the data

In [2]:
# Load the dataset
test_data = pd.read_csv("../data/test.csv")
train_data = pd.read_csv("../data/train.csv")
total_data = pd.concat([train_data, test_data])

print("Number of passengers in train data: ", len(train_data))
train_data.head(5)

Number of passengers in train data:  891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data histograms

In [3]:
"""
fig, axis = plt.subplots(3,3, figsize=(24,10))
total_data["Pclass"].hist(ax = axis[0][0])
axis[0][0].set_title("Pclass")

total_data["Sex"].hist(ax = axis[0][1])
axis[0][1].set_title("Sex")

total_data["Age"].hist(ax = axis[0][2])
axis[0][2].set_title("Age")

total_data["SibSp"].hist(ax = axis[1][0])
axis[1][0].set_title("SibSp")

total_data["Parch"].hist(ax = axis[1][1])
axis[1][1].set_title("Parch")

total_data["Ticket"].hist(ax = axis[1][2])
axis[1][2].set_title("Ticket")

total_data["Fare"].hist(ax = axis[2][0])
axis[2][0].set_title("Fare")

total_data["Cabin"].hist(ax = axis[2][1])
axis[2][1].set_title("Cabin")

total_data["Embarked"].hist(ax = axis[2][2])
axis[2][2].set_title("Embarked")
"""


'\nfig, axis = plt.subplots(3,3, figsize=(24,10))\ntotal_data["Pclass"].hist(ax = axis[0][0])\naxis[0][0].set_title("Pclass")\n\ntotal_data["Sex"].hist(ax = axis[0][1])\naxis[0][1].set_title("Sex")\n\ntotal_data["Age"].hist(ax = axis[0][2])\naxis[0][2].set_title("Age")\n\ntotal_data["SibSp"].hist(ax = axis[1][0])\naxis[1][0].set_title("SibSp")\n\ntotal_data["Parch"].hist(ax = axis[1][1])\naxis[1][1].set_title("Parch")\n\ntotal_data["Ticket"].hist(ax = axis[1][2])\naxis[1][2].set_title("Ticket")\n\ntotal_data["Fare"].hist(ax = axis[2][0])\naxis[2][0].set_title("Fare")\n\ntotal_data["Cabin"].hist(ax = axis[2][1])\naxis[2][1].set_title("Cabin")\n\ntotal_data["Embarked"].hist(ax = axis[2][2])\naxis[2][2].set_title("Embarked")\n'

# Discover bad records

In [4]:
# NAN records
total_data.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

# Fix bad records

## Embarkment

In [5]:
# 2 embarks are missing
total_data[total_data['Embarked'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [6]:
# Find the mode of the embarkment location of females in upper class
filtered = total_data[(total_data.Sex == 'female') & (total_data.Pclass == 1)].groupby('Embarked').count()
filtered['PassengerId']

Embarked
C    71
Q     2
S    69
Name: PassengerId, dtype: int64

In [7]:
# Since the mode is 'C', set the nan's to 'C'
total_data['Embarked'].fillna('C', inplace=True)

## Age

In [8]:
# Look at the correlation between age and the other features
corr_matrix = total_data.corr().abs()
corr_matrix['Age'].sort_values(ascending=False)

Age            1.000000
Pclass         0.408106
SibSp          0.243699
Fare           0.178740
Parch          0.150917
Survived       0.077221
PassengerId    0.028814
Name: Age, dtype: float64

In [9]:
# Since Pclass is the best predictors for age, get the mean age for people in the same class
filter = total_data.groupby(['Pclass']).mean()
filter['Age']

Pclass
1    39.159930
2    29.506705
3    24.816367
Name: Age, dtype: float64

In [10]:
def get_age(Pclass):
    if (Pclass == 1):
        return 39.16
    elif (Pclass == 2):
        return 29.51
    elif (Pclass == 3):
        return 24.82

# Set the ages
total_data['Age'] = total_data.apply(
    lambda row: 
        get_age(row['Pclass']) if np.isnan(row['Age']) else row['Age'],
        axis=1
)

# Fare

In [11]:
# Only one missing fare
total_data[total_data['Fare'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [12]:
# Get the mean price of a 3d class ticket
filter = total_data[total_data['Pclass'] == 3].mean()
filter['Fare']

/tmp/ipykernel_4250/2479175796.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  filter = total_data[total_data['Pclass'] == 3].mean()


13.302888700564973

In [13]:
# Fill the NAN
total_data['Fare'].fillna(13.30, inplace=True)

## Cabin

In [14]:
# Bin the cabins with only their letter code
def bin_cabin(cabin):
    # Get the first letter
    return cabin[0]

total_data['Cabin'] = total_data.apply(
        lambda row: bin_cabin(row['Cabin']) if type(row['Cabin']) == str else 'U',
        axis=1
    )

In [15]:
total_data.Cabin.head(10)

0    U
1    C
2    U
3    C
4    U
5    U
6    E
7    U
8    U
9    U
Name: Cabin, dtype: object

# Feature engineering

In [16]:
# Drop unneeded columns
total_data.drop(['PassengerId', 'Name', 'Ticket'], inplace=True, axis=1)

# Data encoding

In [17]:
# Initialize one-hot encoder
onehot_encoder = OneHotEncoder(sparse=False)

# Define the categorical features that need one-hot encoding
features = ['Pclass', 'Sex', 'Cabin', 'Embarked']

total_df = pd.DataFrame()

for feature in features:
    # One-hot encode
    output = onehot_encoder.fit_transform(total_data[feature].values.reshape(-1,1))

    # Get the number of bits needed for encoding
    nr_encoding_bits = len(output[0])

    # Initialize the column names
    column_names = []
    for idx in range(nr_encoding_bits):
        column_names.append(f'{feature}_{idx}')

    # Initialize the dataframe
    feature_df = pd.DataFrame(columns=column_names)

    # Add the output to the data
    for encoded_row in output:
        # Create a new dataframe with the encoded bits row
        row_df = pd.DataFrame([encoded_row], columns=column_names)

        # Add the new row in the 
        feature_df = pd.concat([feature_df, row_df], ignore_index=True)

    total_df = pd.concat([total_df, feature_df], axis=1)

total_df.head(5)

,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Cabin_7,Cabin_8,Embarked_0,Embarked_1,Embarked_2
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [18]:
# Reset index, otherwise err
total_data.reset_index(inplace=True, drop=True)

# Add the one-hot encoded features to the data
total_data = pd.concat([total_data, total_df], axis=1)

# Remove the original unencoded columns
total_data.drop(features, inplace=True, axis=1)

total_data.head(5)

,Survived,Age,SibSp,Parch,Fare,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,...,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Cabin_7,Cabin_8,Embarked_0,Embarked_1,Embarked_2
0,0.0,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,38.0,1,0,71.2833,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,26.0,0,0,7.9250,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,35.0,1,0,53.1000,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


# Final cleanup

In [19]:
# Initialize a scaler
scaler = StandardScaler()

# Normalize age
normalized_age = scaler.fit_transform(total_data['Age'].values.reshape(-1,1))
total_data['Age'] = normalized_age

# Normalize fare
normalized_fare = scaler.fit_transform(total_data['Fare'].values.reshape(-1,1))
total_data['Fare'] = normalized_fare

In [20]:
# The cabin feature doesn't seem to be a very good predictor
#cabin_features = ['Cabin_0','Cabin_1','Cabin_2','Cabin_3','Cabin_4','Cabin_5','Cabin_6','Cabin_7','Cabin_8']
#total_data.drop(cabin_features, inplace=True, axis=1)

# The SibSp and Parch features dont seem to be very good predictors
#cabin_features = ['SibSp', 'Parch']
#total_data.drop(cabin_features, inplace=True, axis=1)

In [21]:
total_data.corr()

,Survived,Age,SibSp,Parch,Fare,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,...,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Cabin_7,Cabin_8,Embarked_0,Embarked_1,Embarked_2
Survived,1.000000,-0.048926,-0.035322,0.081629,0.257307,0.285904,0.093349,-0.322308,0.543351,-0.543351,...,0.114652,0.150716,0.145321,0.057935,0.016040,-0.026456,-0.316912,0.174718,0.003650,-0.155660
Age,-0.048926,1.000000,-0.198887,-0.131213,0.198334,0.427917,0.006240,-0.375370,-0.053277,0.053277,...,0.192886,0.146836,0.117987,-0.073147,-0.081864,0.032978,-0.309786,0.088426,-0.071262,-0.032862
SibSp,-0.035322,-0.198887,1.000000,0.373587,0.160357,-0.034256,-0.052419,0.072610,0.109609,-0.109609,...,0.048616,-0.015727,-0.027180,-0.008619,0.006015,-0.013247,0.009064,-0.050068,-0.048678,0.075198
Parch,0.081629,-0.131213,0.373587,1.000000,0.221641,-0.013033,-0.010057,0.019521,0.213125,-0.213125,...,0.009601,-0.027385,0.001084,0.020481,0.058325,-0.012304,-0.036806,-0.010287,-0.100943,0.073258
Fare,0.257307,0.198334,0.160357,0.221641,1.000000,0.600091,-0.121212,-0.419864,0.185693,-0.185693,...,0.401429,0.072789,0.073998,-0.037527,-0.022837,0.001187,-0.507327,0.288997,-0.129952,-0.172839
Pclass_0,0.285904,0.427917,-0.034256,-0.013033,0.600091,1.000000,-0.296526,-0.622172,0.107371,-0.107371,...,0.485974,0.275698,0.242963,-0.073083,-0.035441,0.048310,-0.776987,0.331414,-0.166101,-0.187353
Pclass_1,0.093349,0.006240,-0.052419,-0.010057,-0.121212,-0.296526,1.000000,-0.563180,0.028862,-0.028862,...,-0.144104,-0.037929,-0.050210,0.127371,-0.032081,-0.014325,0.176485,-0.136260,-0.121973,0.197973
Pclass_2,-0.322308,-0.375370,0.072610,0.019521,-0.419864,-0.622172,-0.563180,1.000000,-0.116562,0.116562,...,-0.302359,-0.207455,-0.169063,-0.041178,0.056964,-0.030057,0.527614,-0.175057,0.243706,-0.000181
Sex_0,0.543351,-0.053277,0.109609,0.213125,0.185693,0.107371,0.028862,-0.116562,1.000000,-1.000000,...,0.077473,0.057396,0.040340,0.006655,0.083285,-0.020558,-0.137396,0.071447,0.088651,-0.119504
Sex_1,-0.543351,0.053277,-0.109609,-0.213125,-0.185693,-0.107371,-0.028862,0.116562,-1.000000,1.000000,...,-0.077473,-0.057396,-0.040340,-0.006655,-0.083285,0.020558,0.137396,-0.071447,-0.088651,0.119504


# Save the data

In [22]:
# Split the total data in train and test dataset
output_train = total_data[total_data.Survived.isna() == False]
output_test = total_data[total_data.Survived.isna()]

# Drop the 'Survived' column in the test data
output_test.drop('Survived', inplace=True, axis = 1)

/tmp/ipykernel_4250/2309964367.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_test.drop('Survived', inplace=True, axis = 1)


In [23]:
output_train.to_csv('../data/clean/cleaned_train.csv')
output_test.to_csv('../data/clean/cleaned_test.csv')